# API를 활용하여 아파트매매 실거래 데이터 가져오기

# 📌기초 - 데이터 수집

### 라이브러리 불러오기

In [1]:
from xml.etree import ElementTree as ET
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from time import strftime
# import numpy as np
# import urllib

## 활용 데이터

공공데이터포털 > 국토교통부_아파트매매 실거래 상세 자료 <br>
https://www.data.go.kr/data/15057511/openapi.do

### 요청변수(Request Parameter)

In [ ]:
info_url = 'https://www.data.go.kr/data/15057511/openapi.do'

table = pd.read_html(info_url)
request_parameter = table[4]
request_parameter

### 출력결과(Response Element)

In [ ]:
response_element = table[5]
response_element

### (참고) 샘플 코드

In [ ]:
import requests

url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev'
key = ' - - api key 입력 - - '
params ={'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '10', 'LAWD_CD' : '11110', 'DEAL_YMD' : '202301' }

response = requests.get(url, params=params)
print(response.content)

## 코드 리뷰

### 쿼리 값 세팅

In [17]:
key =  ' - - decoding key 입력 - - '
LAWD_CD = '11200' # 성동구 지역코드
DEAL_YMD = '202301'

# f-string 방식
query = f'?serviceKey={key}&pageNo=1&numOfRows=99999&LAWD_CD={LAWD_CD}&DEAL_YMD={DEAL_YMD}'

In [18]:
# dictionary 방식
params ={'serviceKey' : key, 'numOfRows' : '99999', 'LAWD_CD' : LAWD_CD , 'DEAL_YMD' : DEAL_YMD}

### url 세팅

In [19]:
url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev'
rq_url = url + query

### GET OK 200 확인

In [20]:
# f-string
# response = requests.get(rq_url)

# dictionary
response = requests.get(url, params=params)

response

<Response [200]>

### 데이터 수집
<b><방식><br>
- 파싱 된 결과를 가져온다. = result <br>
- 가져와야하는 문서의 태그 형태는 \<items>안에 \<item>으로 개별 거래내역이 들어 있고, 그 안에 컬럼명이 태그로 들어가 있다. <br>
    ```ex. <item><거래금액>   155,000</거래금액><거래유형>중개거래</거래유형><건축년도>2016</건축년도> … <해제여부> </해제여부></item> ```

<b><컬럼명 수집><br>
- 임의로 한 개의 item을 .find_all() 메서드로 태그와 내용을 모두 가져온 뒤 .name 메서드로 태그명을 모두 가져온다.

<b><내용 수집><br>
- 내용은 결과.태그명.text 또는 결과.태그명.string 으로 태그 내 텍스트를 가져온다. <br>
- 위 코드에 for문으로 컬럼명을 순차적으로 반복시켜주면 모든 내용을 가져올 수 있다.
    
<b><수집 데이터 정리><br>
- 위 방법으로 수집하면, 개별 거래내역별로 묶이는게 아니라 전체 거래내역이 컬럼순으로 데이터가 한데 묶이는 문제가 생긴다.
- 내용이 없는 부분(ex. 해제여부)도 태그는 있기 때문에, 누락된 칸이 없다. (아마도)
- 그래서 range의 step 속성으로 수집된 거래내역 숫자만큼씩 데이터를 뛰어넘어서 묶어주는 방식으로 정리했다.
- 비효율적이고 혹시나 누락데이터가 있을 땐 다 망가지는 방법이고, 태그명을 따라 내용을 따로 저장해주는게 가장 좋을 것 같은데 아는 범위내에서는 아직 이렇게 밖에 해결을 못했다😓

### 컬럼 매핑

In [26]:
# 방법 1
result = bs(response.text, features='xml')
items = result.findAll('item')

# # 방법 2 : soup의 내용은 response.text 결과와 같지만 타입이 다르다.
# soup = bs(response.content, 'lxml-xml')
# items = soup.findAll('item')

# 컬럼명 찾기
columns = items[0].find_all()
columns[0].name

cols = []

for i in range(0, len(columns)):
    cols.append(columns[i].name)

print(cols)

['거래금액', '거래유형', '건축년도', '년', '도로명', '도로명건물본번호코드', '도로명건물부번호코드', '도로명시군구코드', '도로명일련번호코드', '도로명코드', '법정동', '법정동본번코드', '법정동부번코드', '법정동시군구코드', '법정동읍면동코드', '법정동지번코드', '아파트', '월', '일', '일련번호', '전용면적', '중개사소재지', '지번', '지역코드', '층', '해제사유발생일', '해제여부']


.name 메서드는 BeautifulSoup 메서드 <br>
관련 내용은 BS4 공식 문서의 <.contents and .children > 부분 참고 <br>
https://www.crummy.com/software/BeautifulSoup/bs4/doc/#contents-and-children

### 내용 가져오기

In [27]:
# 문서 내 모든 텍스트 값 가져오기
all_txt = []

for c in range(0,len(cols)):
    col = cols[c]
    for a in result.select(col):
        all_txt.append(a.text.strip())
        
print(all_txt)

['155,000', '83,000', '129,000', '186,000', '133,000', '124,000', '101,000', '중개거래', '중개거래', '중개거래', '중개거래', '중개거래', '중개거래', '중개거래', '2016', '2000', '2003', '2007', '2012', '2016', '1990', '2023', '2023', '2023', '2023', '2023', '2023', '2023', '왕십리로', '행당로', '고산자로', '금호로', '매봉길', '매봉길', '서울숲길', '00410', '00079', '00164', '00015', '00015', '00050', '00025', '00000', '00000', '00000', '00000', '00000', '00000', '00000', '11200', '11200', '11200', '11200', '11200', '11200', '11200', '00', '01', '04', '03', '02', '00', '01', '3005011', '3103006', '3005030', '3103001', '4109229', '4109229', '4109323', '하왕십리동', '행당동', '행당동', '금호동4가', '옥수동', '옥수동', '성수동1가', '1070', '0347', '0349', '0340', '0561', '0528', '0676', '0000', '0000', '0000', '0000', '0000', '0000', '0005', '11200', '11200', '11200', '11200', '11200', '11200', '11200', '10200', '10700', '10700', '11200', '11300', '11300', '11400', '1', '1', '1', '1', '1', '1', '1', '센트라스', '대림e-편한세상', '서울숲 한신 더 휴', '서울숲푸르지오', '래미안 옥수 리버젠', '옥수파크힐스1

In [28]:
# 개별 거래내역별로 묶기
sorted_txt = []

for i in range(len(result.items)):
    temp = []
    a = list(range(i, len(all_txt), len(result.items)))
    for j in a:
        temp.append(all_txt[j])
    sorted_txt.append(temp)
 
print(sorted_txt)

[['155,000', '중개거래', '2016', '2023', '왕십리로', '00410', '00000', '11200', '00', '3005011', '하왕십리동', '1070', '0000', '11200', '10200', '1', '센트라스', '1', '11', '11200-3120', '84.96', '서울 성동구', '1070', '11200', '21', '', ''], ['83,000', '중개거래', '2000', '2023', '행당로', '00079', '00000', '11200', '01', '3103006', '행당동', '0347', '0000', '11200', '10700', '1', '대림e-편한세상', '1', '9', '11200-18', '59.96', '서울 성동구', '347', '11200', '6', '', ''], ['129,000', '중개거래', '2003', '2023', '고산자로', '00164', '00000', '11200', '04', '3005030', '행당동', '0349', '0000', '11200', '10700', '1', '서울숲 한신 더 휴', '1', '11', '11200-24', '114.97', '서울 성동구', '349', '11200', '14', '', ''], ['186,000', '중개거래', '2007', '2023', '금호로', '00015', '00000', '11200', '03', '3103001', '금호동4가', '0340', '0000', '11200', '11200', '1', '서울숲푸르지오', '1', '14', '11200-2820', '84.87', '서울 성동구', '340', '11200', '17', '', ''], ['133,000', '중개거래', '2012', '2023', '매봉길', '00015', '00000', '11200', '02', '4109229', '옥수동', '0561', '0000', '11200', '1

In [30]:
# 컬럼과 내용 병합
df = pd.DataFrame(sorted_txt, columns = cols)
display(df)
df.shape

,거래금액,거래유형,건축년도,년,도로명,도로명건물본번호코드,도로명건물부번호코드,도로명시군구코드,도로명일련번호코드,도로명코드,...,월,일,일련번호,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
0,"155,000",중개거래,2016,2023,왕십리로,00410,00000,11200,00,3005011,...,1,11,11200-3120,84.96,서울 성동구,1070,11200,21,,
1,"83,000",중개거래,2000,2023,행당로,00079,00000,11200,01,3103006,...,1,9,11200-18,59.96,서울 성동구,347,11200,6,,
2,"129,000",중개거래,2003,2023,고산자로,00164,00000,11200,04,3005030,...,1,11,11200-24,114.97,서울 성동구,349,11200,14,,
3,"186,000",중개거래,2007,2023,금호로,00015,00000,11200,03,3103001,...,1,14,11200-2820,84.87,서울 성동구,340,11200,17,,
4,"133,000",중개거래,2012,2023,매봉길,00015,00000,11200,02,4109229,...,1,9,11200-2980,59.25,서울 성동구,561,11200,18,,
5,"124,000",중개거래,2016,2023,매봉길,00050,00000,11200,00,4109229,...,1,19,11200-3114,59.91,서울 성동구,528,11200,6,,
6,"101,000",중개거래,1990,2023,서울숲길,00025,00000,11200,01,4109323,...,1,2,11200-61,84.73,서울 성동구,676-5,11200,13,,


(7, 27)

In [31]:
# 컬럼을 편집하고 싶을 경우
# edit_cols = ['년','월','일','거래유형','지역코드','도로명', '법정동','지번','아파트','층','건축년도','전용면적','거래금액']
# df = df[edit_cols]

# 컬럼을 특정 데이터로 정렬하고 싶을 경우
# df.sort_values(['층'])

### 파일 저장

In [ ]:
file_name = 'API_LandCost_' + strftime('%Y-%m-%d-%H-%M') + '.csv'
df.to_csv(file_name, index=False)
print(file_name)
pd.read_csv(file_name)

# 📌심화 - 데이터 분석

### 숫자 데이터 형 변환

In [33]:
# int 타입이어야 하는 컬럼 : 거래금액, 건축년도, 년, 월, 일, 층
# float 타입이어야 하는 컬럼 : 전용면적

In [34]:
#public data reader 코드 중 컬럼 내 데이터 타입 변환 참고
int_cols = ['거래금액', '건축년도', '년', '월', '일', '층']
float_cols = ['전용면적']

for col in int_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col].apply(
            lambda x: x.strip().replace(",", "") if x is not None and not pd.isnull(x) else x)).astype("Int64")
for col in float_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col])

In [36]:
# 숫자형으로 잘 변환됐는지 확인

print(type(df.거래금액[0]))
print(df.거래금액[0] + df.거래금액[1])

<class 'numpy.int64'>
238000


### 단위 변경(m² > 평)

In [37]:
pyong = [round(var/ 3.305785, 1) for var in df['전용면적']]
df['평형'] = pyong

In [38]:
df[['년', '월', '일', '아파트', '층', '건축년도', '전용면적', '평형', '거래금액']]

,년,월,일,아파트,층,건축년도,전용면적,평형,거래금액
0,2023,1,11,센트라스,21,2016,84.96,25.7,155000
1,2023,1,9,대림e-편한세상,6,2000,59.96,18.1,83000
2,2023,1,11,서울숲 한신 더 휴,14,2003,114.97,34.8,129000
3,2023,1,14,서울숲푸르지오,17,2007,84.87,25.7,186000
4,2023,1,9,래미안 옥수 리버젠,18,2012,59.25,17.9,133000
5,2023,1,19,옥수파크힐스101동~116동,6,2016,59.91,18.1,124000
6,2023,1,2,서울숲성수현대,13,1990,84.73,25.6,101000


### 평당 가격 계산

In [40]:
cost_per_pyong = round(df['거래금액'] / df['평형'])
df['평당금액'] = cost_per_pyong

In [41]:
df[['년', '월', '일', '아파트', '층', '건축년도', '전용면적', '평형', '거래금액', '평당금액']]

,년,월,일,아파트,층,건축년도,전용면적,평형,거래금액,평당금액
0,2023,1,11,센트라스,21,2016,84.96,25.7,155000,6031.0
1,2023,1,9,대림e-편한세상,6,2000,59.96,18.1,83000,4586.0
2,2023,1,11,서울숲 한신 더 휴,14,2003,114.97,34.8,129000,3707.0
3,2023,1,14,서울숲푸르지오,17,2007,84.87,25.7,186000,7237.0
4,2023,1,9,래미안 옥수 리버젠,18,2012,59.25,17.9,133000,7430.0
5,2023,1,19,옥수파크힐스101동~116동,6,2016,59.91,18.1,124000,6851.0
6,2023,1,2,서울숲성수현대,13,1990,84.73,25.6,101000,3945.0


### (참고) XML 파일 처리
https://codealone.tistory.com/77

In [56]:
tree = ET.fromstring(response.content)
# ET.dump(tree) # .dump : xml 요소의 내용 출력 (response.text와 형태 비슷)
ET.indent(tree, space = ' ') # 들여쓰기 처리(가독성)
ET.dump(tree)

In [53]:
# 파일 내 태그트리 구성형태대로 리스트화 되어 있음
# response[header[resultCode, resultMsg],body[items[item, item …], …]

items = tree[1][0] # body 태그 안의 items의 위치

# 컬럼과 내용 저장
columns = [i.tag for i in items[1]]
items_list = [[i.text for i in item] for item in items]

In [54]:
df = pd.DataFrame(items_list, columns = columns)